In [17]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from collections import OrderedDict

import re
from time import sleep
from random import randint

In [27]:
def fix_dollar(x):
  x=str(x)
  x=x.replace(')', '') 
  x=x.replace('(', '') 
  x=x.replace('$', '')
  if 'M' in x:
    x=float(x.replace('M', ''))
    x=x*1000000
    return x
  if 'B' in x:
    x=float(x.replace('B', ''))
    x=x*1000000000
  return x

def fix_eth(x):
  x=str(x)
  x=x.replace('Ξ', '')
  x=x.replace('<', '')
  x=x.replace(',', '')
  if 'K' in x:
    x=float(x.replace('K', ''))
    x=x*1000
    return x
  return x

In [29]:
last_id = 9999
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'From': 'danyal1921@gmail.com'
}

i= 7961
while i<= last_id:
  page = requests.get("https://www.larvalabs.com/cryptopunks/details/{}".format(i), headers)
  soup = BeautifulSoup(page.text, "html.parser")

  tr_hist = soup.find_all('table')
  df = pd.read_html(str(tr_hist))[0]
  df['punk_nr']=i
  df['Txn'] = pd.to_datetime(df['Txn'])
    
  if df.Amount.isna().all() ==True: #For cryptopunks with no bidding history, split the empty amount column and fetch the df
    df.Amount.fillna('', inplace=True)
    df[['price_in_eth', 'price_in_dollars']] = df['Amount'].str.split('-', n=1, expand=True).reindex([1,0], axis=1)
    df.drop('Amount', axis=1, inplace=True)
    df.to_csv('transaction.csv', header=None, mode='a')
  
  else: #For cryptopunks with bidding history, plit the string and convert the digits to float, then fetch the df
    df[['price_in_eth', 'price_in_dollars']] = df['Amount'].str.split(' ', 1, expand=True)
    df.price_in_eth = df.price_in_eth.apply(fix_eth)
    df.price_in_dollars = df.price_in_dollars.apply(fix_dollar)
    df.drop('Amount', axis=1, inplace=True)
    df.to_csv('transaction.csv', header=None, mode='a')

  newlist=[[]]
  newlist[0].append(i)
  name = soup.find('div', class_='col-md-10 col-md-offset-1 col-xs-12')
  newlist[0].append(name.h4.a.text)
    
  for y in soup.find_all('div', class_='col-md-4')[1:]:
    newlist[0].append(y.a.get_text())

  x = pd.DataFrame(newlist)
  x.to_csv('accessories.csv', header=None, mode='a', index=False)

  sleep(randint(2,10))
  i += 1